# Data Imputation for Wild Meat Nutritional Composition

This notebook details the process of imputing missing nutritional data for the study "Comprehensive Nutritional Composition of Wild Meat: A Systematic Review Using Data Imputation with Artificial Intelligence". 

The key steps involved are:
1.  **Loading Processed Data**: Import the cleaned dataset from the previous data preparation phase.
2.  **Preparing Data for Imputation**: One-hot encode categorical variables to make them suitable for various imputation algorithms.
3.  **Cross-Validation of Imputation Methods**: Systematically evaluate a range of imputation techniques for each nutrient column using a leave-one-out cross-validation approach. This helps in selecting the most appropriate imputer for each specific nutrient based on performance metrics.
4.  **Full Dataset Imputation**: Apply the best-performing imputer (identified through cross-validation) to each nutrient column to fill in missing values.
5.  **Post-Imputation Processing**: Ensure imputed values are non-negative and save the imputed dataset.
6.  **Generating Outputs**: Create user-friendly versions of the imputed data, summary statistics, and visualizations for analysis and reporting.

## 1. Setup and Library Imports

Import necessary libraries for data manipulation, various imputation algorithms (from `fancyimpute` and `sklearn`), performance metrics, and parallel processing.

In [ ]:
import datetime
import time
import warnings
from concurrent.futures import ProcessPoolExecutor, as_completed
from functools import partial

import numpy as np
import pandas as pd

# Imputation libraries
from fancyimpute import BiScaler, IterativeSVD, MatrixFactorization, NuclearNormMinimization, SoftImpute

# Note: IterativeImputer from fancyimpute is often less maintained; sklearn's is preferred
from sklearn.impute import IterativeImputer as IterativeImputer_sklearn
from sklearn.impute import KNNImputer, SimpleImputer

# Machine learning models and metrics
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, r2_score
from tqdm import tqdm  # For progress bars

# Custom helper function for Symmetric Mean Absolute Percentage Error (SMAPE)
from helpers import symmetric_mape

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")
# Pandas display options
pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 500)

## 2. Data Loading and Preparation for Imputation

### 2.1. Define Columns for Imputation
Specify the list of nutrient columns that have missing values and require imputation.

In [2]:
# List of nutrient columns to be targeted for imputation
cols_impute = ["fe", "mn", "se", "zn", "k", "mg", "na", "ptn", "lip", "w3", "w6"]

### 2.2. Load Processed Data
Load the dataset that was cleaned and prepared in the `01-prepare_data.ipynb` notebook.

In [3]:
data = pd.read_csv("data/input_data_processed.csv")
# Display the first few rows of the loaded data (optional)
data

,taxon_tfmed,genus,classificacao,pais_coleta,fe,mn,se,zn,k,mg,na,ptn,lip,w3,w6,partes_tfmed,classificacao_cat,pais_coleta_cat,partes_tfmed_cat,taxon_tfmed_cat,genus_cat
0,SS,sus,mamifero,Alemanha,0.001900,NaN,0.000013,0.002400,NaN,NaN,NaN,22.5000,2.10,0.077700,0.571200,Musculo,1,1,0,24,21
1,CC1,capreolus,mamifero,Alemanha,0.003210,NaN,0.000004,0.002350,NaN,NaN,NaN,23.5000,1.00,0.110000,0.266000,Musculo,1,1,0,4,5
2,EQ,equus,mamifero,África do Sul,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.2900,1.47,0.000032,0.000005,Musculo,1,14,0,12,11
3,CE,cervus,mamifero,Polônia,NaN,0.000076,0.000005,0.004950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Musculo,1,13,0,7,6
4,CE,cervus,mamifero,Polônia,NaN,0.000396,0.000007,0.003300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Visceras,1,13,1,7,6
5,CE,cervus,mamifero,Polônia,NaN,0.000218,0.000132,0.004290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Visceras,1,13,1,7,6
6,CE,cervus,mamifero,EUA,0.015131,0.000214,NaN,0.002040,8.200000e-08,0.015988,NaN,NaN,NaN,NaN,NaN,Visceras,1,4,1,7,6
7,CE,cervus,mamifero,EUA,0.013225,0.000211,NaN,0.001924,1.400000e-08,0.016113,NaN,NaN,NaN,NaN,NaN,Visceras,1,4,1,7,6
8,SR,scolopax,ave,Italia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.0000,2.75,0.000197,1.570000,Musculo,0,7,0,23,19
9,SR,scolopax,ave,Italia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.0500,2.60,0.000175,1.040000,Musculo,0,7,0,23,19


In [4]:
# Verify column names
data.columns

Index(['taxon_tfmed', 'genus', 'classificacao', 'pais_coleta', 'fe', 'mn',
       'se', 'zn', 'k', 'mg', 'na', 'ptn', 'lip', 'w3', 'w6', 'partes_tfmed',
       'classificacao_cat', 'pais_coleta_cat', 'partes_tfmed_cat',
       'taxon_tfmed_cat', 'genus_cat'],
      dtype='object')

### 2.3. One-Hot Encode Categorical Features
Many imputation algorithms work best with numerical data. Categorical features (`taxon_tfmed`, `genus`, `classificacao`, `pais_coleta`, `partes_tfmed`) are converted into a numerical format using one-hot encoding. This creates new binary (0 or 1) columns for each category within these features.

In [5]:
data_onehot = data.copy()
# Select relevant columns including categorical ones for one-hot encoding and nutrient columns
data_onehot = data_onehot[["taxon_tfmed", "genus", "classificacao", "pais_coleta", "fe", "mn", "se", "zn", "k", "mg", "na", "ptn", "lip", "w3", "w6", "partes_tfmed"]]

# Perform one-hot encoding for specified categorical columns
data_onehot = pd.get_dummies(data_onehot, columns=["taxon_tfmed", "genus", "classificacao", "pais_coleta", "partes_tfmed"])

# Convert the entire DataFrame to float type, as imputation algorithms expect numerical input
data_onehot = data_onehot.astype(float)

# Display the one-hot encoded DataFrame (optional)
data_onehot

,fe,mn,se,zn,k,mg,na,ptn,lip,w3,w6,taxon_tfmed_AA,taxon_tfmed_AP1,taxon_tfmed_AP2,taxon_tfmed_BC,taxon_tfmed_CC1,taxon_tfmed_CC2,taxon_tfmed_CD,taxon_tfmed_CE,taxon_tfmed_CP1,taxon_tfmed_CP2,taxon_tfmed_CY,taxon_tfmed_DL,taxon_tfmed_EQ,taxon_tfmed_MA,taxon_tfmed_OH,taxon_tfmed_OV,taxon_tfmed_PA,taxon_tfmed_PD,taxon_tfmed_PE,taxon_tfmed_PQ,taxon_tfmed_PS,taxon_tfmed_PU,taxon_tfmed_SC,taxon_tfmed_SR,taxon_tfmed_SS,taxon_tfmed_ST,taxon_tfmed_TP1,taxon_tfmed_TP2,taxon_tfmed_TT,genus_agouti,genus_alces,genus_anas,genus_branta,genus_caiman,genus_capreolus,genus_cervus,genus_chelonoidis,genus_columba,genus_cuniculus,genus_dasyprocta,genus_equus,genus_manzama,genus_mazama,genus_odocoileus,genus_pecari,genus_peltocephalus,genus_phacochoerus,genus_podocnemis,genus_scolopax,genus_streptopelia,genus_sus,genus_syncerus,genus_tayassu,genus_turdus,classificacao_ave,classificacao_mamifero,classificacao_reptil,pais_coleta_Africa do Sul,pais_coleta_Alemanha,pais_coleta_Brasil,pais_coleta_Canadá,pais_coleta_EUA,pais_coleta_Espanha,pais_coleta_Estados Unidos da América,pais_coleta_Italia,pais_coleta_Itália,pais_coleta_Letónia,pais_coleta_Nova Zelândia,pais_coleta_Peru,pais_coleta_Polônia,pais_coleta_África do Sul,pais_coleta_África do sul,partes_tfmed_Musculo,partes_tfmed_Visceras
0,0.001900,NaN,0.000013,0.002400,NaN,NaN,NaN,22.5000,2.10,0.077700,0.571200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.003210,NaN,0.000004,0.002350,NaN,NaN,NaN,23.5000,1.00,0.110000,0.266000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.2900,1.47,0.000032,0.000005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,NaN,0.000076,0.000005,0.004950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,NaN,0.000396,0.000007,0.003300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
5,NaN,0.000218,0.000132,0.004290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
6,0.015131,0.000214,NaN,0.002040,8.200000e-08,0.015988,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,0.013225,0.000211,NaN,0.001924,1.400000e-08,0.016113,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

## 3. Cross-Validation Framework for Imputation Method Selection

To objectively select the best imputation method for each nutrient, a leave-one-out cross-validation (LOOCV) approach is implemented. For each non-missing value in a nutrient column:
1.  The value is temporarily removed (set to NaN).
2.  An imputation algorithm is used to predict this removed value.
3.  The predicted value is compared against the original (real) value.
This process is repeated for all non-missing values and for each imputation method considered. Performance metrics (R², RMSE, MAE, MAPE, SMAPE) are then calculated to evaluate each imputer's accuracy for that specific nutrient.

In [6]:
def get_idx_non_null_rows(dataframe, column):
    """Helper function to get indices of rows where the specified column is not null."""
    return dataframe[dataframe[column].notnull()].index


def impute_cv(df_input, col_name, imputer_name):
    """Performs leave-one-out cross-validation for a given imputer on a specific column.

    Args:
        df_input (pd.DataFrame): The one-hot encoded DataFrame with missing values.
        col_name (str): The name of the column (nutrient) to perform CV on.
        imputer_name (str): The name of the imputer to use.

    Returns:
        list: A list of tuples, each containing (imputer_name, col_name, real_value, predicted_value, time_elapsed).
    """
    df = df_input.copy()  # Work on a copy to avoid modifying the original DataFrame
    results = []
    try:
        col_idx = df.columns.get_loc(col_name)  # Get the integer index of the target column
        # Convert DataFrame to NumPy array for fancyimpute compatibility and performance
        input_df_np = df.values
        idx_non_null_rows = get_idx_non_null_rows(df, col_name)

        # Iterate over each non-null value in the target column for LOOCV
        for row_idx in idx_non_null_rows:
            start_time = time.time()
            # Store the real value and temporarily set it to NaN
            real_value = input_df_np[row_idx, col_idx].copy()
            input_df_np_cv = input_df_np.copy()  # Create a fresh copy for each CV iteration
            input_df_np_cv[row_idx, col_idx] = np.nan

            # Initialize the specified imputer
            if imputer_name.startswith("KNN"):
                k = int(imputer_name.split("_")[1])
                imputer = KNNImputer(n_neighbors=k)
            elif imputer_name == "MatrixFactorization":
                imputer = MatrixFactorization(verbose=False, min_value=0)  # Added min_value based on context
            elif imputer_name == "NuclearNormMinimization":
                imputer = NuclearNormMinimization(verbose=False, min_value=0)
            elif imputer_name == "SoftImpute":
                imputer = SoftImpute(verbose=False, min_value=0)
            elif imputer_name == "BiScaler+SoftImpute":  # This combination might require careful handling
                imputer = SoftImpute(verbose=False, min_value=0)
            elif imputer_name == "IterativeSVD":
                imputer = IterativeSVD(verbose=False, min_value=0)
            elif imputer_name == "IterativeImputer_sklearn":
                imputer = IterativeImputer_sklearn(verbose=0, min_value=0, random_state=314)
            elif imputer_name == "IterativeImputer_sklearn_LinearRegression":
                imputer = IterativeImputer_sklearn(verbose=0, estimator=LinearRegression(), min_value=0, random_state=314)
            # SimpleImputers are generally not used in a multivariate CV like this but included for completeness if tested
            elif imputer_name == "SimpleImputerMean":
                imputer = SimpleImputer(strategy="mean")  # verbose is not an arg for SimpleImputer
            elif imputer_name == "SimpleImputerMedian":
                imputer = SimpleImputer(strategy="median")
            else:
                raise ValueError(f"Unknown imputer: {imputer_name}")

            # Perform imputation
            if imputer_name == "BiScaler+SoftImpute":
                # Apply BiScaler before SoftImpute
                scaled_matrix = BiScaler(verbose=False).fit_transform(input_df_np_cv)
                imputed_matrix = imputer.fit_transform(scaled_matrix)
            else:
                imputed_matrix = imputer.fit_transform(input_df_np_cv)

            predicted_value = imputed_matrix[row_idx, col_idx]
            end_time = time.time()
            time_elapsed = end_time - start_time
            results.append((imputer_name, col_name, real_value, predicted_value, time_elapsed))

            # Restore the original value in the NumPy array for the next iteration (though input_df_np_cv is fresh each time)
            # This line is not strictly necessary due to `input_df_np_cv = input_df_np.copy()` inside the loop
            # input_df_np[row_idx, col_idx] = real_value

        return results
    except Exception as e:
        print(f"Error during CV for {col_name} with {imputer_name}: {e}")
        return [(imputer_name, col_name, np.nan, np.nan, np.nan)]  # Return placeholder for error


def metrics_impute_cv(results):
    """Calculates performance metrics from the cross-validation results."""
    results = [i for i in results if i is not None and not np.isnan(i[2]) and not np.isnan(i[3])]  # Filter out None or NaN results
    if not results:
        # Return a DataFrame with NaNs if no valid results
        return pd.DataFrame({"imputer_name": [None], "col_name": [None], "r2": [np.nan], "rmse": [np.nan], "mae": [np.nan], "mape": [np.nan], "smape": [np.nan], "mae_std_ratio": [np.nan], "mean_time": [np.nan], "real_values": [[]], "predicted_values": [[]]})

    results_df = pd.DataFrame(results, columns=["imputer_name", "col_name", "real_value", "predicted_value", "time_elapsed"])
    imputer_name = results_df["imputer_name"].unique()[0]
    col_name = results_df["col_name"].unique()[0]
    mean_time = results_df["time_elapsed"].mean()
    real_values = results_df["real_value"].values
    predicted_values = results_df["predicted_value"].values

    # Calculate metrics
    r2 = r2_score(real_values, predicted_values)
    rmse = mean_squared_error(real_values, predicted_values, squared=False)  # squared=False for RMSE
    mae = mean_absolute_error(real_values, predicted_values)
    # MAPE can be problematic if real_values contain zeros, SMAPE is more robust
    mape = mean_absolute_percentage_error(real_values, predicted_values)
    smape = symmetric_mape(real_values, predicted_values)
    mae_std = mae / (results_df["real_value"].std() if results_df["real_value"].std() != 0 else np.nan)  # Avoid division by zero

    dataframe_return = pd.DataFrame(
        {"imputer_name": [imputer_name], "col_name": [col_name], "r2": [r2], "rmse": [rmse], "mae": [mae], "mape": [mape], "smape": [smape], "mae_std_ratio": [mae_std], "mean_time": [round(mean_time, 4)], "real_values": [real_values], "predicted_values": [predicted_values]}
    )
    return dataframe_return

In [ ]:
# Example run of the CV process for 'fe' column with KNN_3 imputer
res = impute_cv(data_onehot, "fe", "KNN_3")
metrics_impute_cv(res)

,imputer_name,col_name,r2,rmse,mae,mape,smape,mae_std_ratio,mean_time,real_values,predicted_values
0,KNN_3,fe,0.374481,0.004399,0.002799,1.574901,65.486241,0.493079,0.0054,"[0.0019, 0.00321, 0.015131, 0.013225, 0.0038, ...","[0.0012613333333333333, 0.002369, 0.0055323333..."


Test the CV framework with a single imputer and column.

## 4. Imputer Evaluation using Cross-Validation

A list of multivariate imputation methods is defined. The cross-validation process is then run in parallel for each combination of nutrient column and imputer to efficiently evaluate their performance.

### 4.1. Define List of Imputers to Evaluate

In [7]:
# List of imputer names to be evaluated
list_imputers_names_multi = [
    "KNN_3",
    "KNN_4",
    "KNN_5",
    "KNN_6",
    "KNN_7",
    "KNN_8",
    "KNN_9",
    "KNN_10",
    "KNN_11",
    "KNN_12",
    "KNN_13",
    "KNN_14",
    "KNN_15",
    "MatrixFactorization",
    "SoftImpute",
    "BiScaler+SoftImpute",
    "IterativeSVD",
    "IterativeImputer_sklearn",
    "IterativeImputer_sklearn_LinearRegression",
]

### 4.2. Run Cross-Validation in Parallel

In [8]:
cols_to_impute = ["fe", "mn", "se", "zn", "k", "mg", "na", "ptn", "lip", "w3", "w6"]


# Function to wrap impute_cv and metrics_impute_cv for parallel execution
def impute_col(col, imputer_name, df_input):
    """Processes a single column with a single imputer and returns metrics."""
    try:
        # print(f"Starting: {col} - {imputer_name}") # For debugging
        res_cv = impute_cv(df_input, col, imputer_name)
        metrics_df = metrics_impute_cv(res_cv)
        print(f"Completed: {col} - {imputer_name} - {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        return metrics_df
    except Exception as e:
        print(f"Error in impute_col for {col} with {imputer_name}: {e}")
        # Return a DataFrame with NaNs or an identifier for the error
        return pd.DataFrame({"imputer_name": [imputer_name], "col_name": [col], "r2": [np.nan], "rmse": [np.nan], "mae": [np.nan], "mape": [np.nan], "smape": [np.nan], "mae_std_ratio": [np.nan], "mean_time": [np.nan], "real_values": [[]], "predicted_values": [[]]})


# Use functools.partial to create a version of impute_col with df_input fixed
impute_col_onehot_partial = partial(impute_col, df_input=data_onehot)

In [ ]:
results_onehot_cv = []  # Store results from cross-validation

# Use ProcessPoolExecutor for parallel processing
with ProcessPoolExecutor() as executor:
    # Create a list of future objects for each task (imputing a column with an imputer)
    futures = [executor.submit(impute_col_onehot_partial, col, imputer_name) for col in cols_to_impute for imputer_name in list_imputers_names_multi]

    # Process futures as they complete
    for future in tqdm(as_completed(futures), total=len(futures), desc="Imputation CV Progress"):
        result = future.result()
        if result is not None:
            results_onehot_cv.append(result)

# Concatenate all result DataFrames into a single DataFrame
df_results_onehot_cv = pd.concat(results_onehot_cv).reset_index(drop=True)

se - KNN_5 - 2023-10-06 09:51:45.473348
se - KNN_6 - 2023-10-06 09:51:45.525302
se - KNN_7 - 2023-10-06 09:51:46.023086se - KNN_4 - 2023-10-06 09:51:46.172514

se - KNN_8 - 2023-10-06 09:51:47.214528
se - KNN_12 - 2023-10-06 09:51:47.691743
se - KNN_3 - 2023-10-06 09:51:47.906473
fe - KNN_15 - 2023-10-06 09:51:47.939816
fe - KNN_10 - 2023-10-06 09:51:48.563898
mn - KNN_5 - 2023-10-06 09:51:48.990990
se - KNN_10 - 2023-10-06 09:51:49.527010fe - KNN_9 - 2023-10-06 09:51:49.891936
se - KNN_9 - 2023-10-06 09:51:50.030360se - KNN_11 - 2023-10-06 09:51:50.025457

se - KNN_15 - 2023-10-06 09:51:49.968513fe - KNN_8 - 2023-10-06 09:51:50.125522


mn - KNN_14 - 2023-10-06 09:51:50.246809se - KNN_14 - 2023-10-06 09:51:50.100333
mn - KNN_6 - 2023-10-06 09:51:50.235278

fe - KNN_13 - 2023-10-06 09:51:51.303369
se - KNN_13 - 2023-10-06 09:51:51.421383
fe - KNN_5 - 2023-10-06 09:51:51.629754
mn - KNN_4 - 2023-10-06 09:51:51.718811
fe - KNN_6 - 2023-10-06 09:51:52.644239
mn - KNN_15 - 2023-10-06 09:51

### 4.3. Analyze Cross-Validation Results
Display the top-performing imputers for each nutrient column based on SMAPE (Symmetric Mean Absolute Percentage Error) and RMSE (Root Mean Squared Error). SMAPE is often preferred as it's less sensitive to outliers than MAPE and symmetric.

In [ ]:
# Group results by column name and show the top 2 imputers based on SMAPE and RMSE
df_results_onehot_cv.groupby("col_name").apply(lambda x: x.sort_values(["smape", "rmse"]).head(2)).reset_index(drop=True)

,imputer_name,col_name,r2,rmse,mae,mape,smape,mae_std_ratio,mean_time,real_values,predicted_values
0,IterativeImputer_sklearn,fe,0.648056,0.003300,0.001860,1.133531e+00,61.964051,0.327668,232.9856,"[0.0019, 0.00321, 0.015131, 0.013225, 0.0038, ...","[0.003862832892899064, 0.0031234627212915825, ..."
1,KNN_3,fe,0.374481,0.004399,0.002799,1.574901e+00,65.486241,0.493079,0.6846,"[0.0019, 0.00321, 0.015131, 0.013225, 0.0038, ...","[0.0012613333333333333, 0.002369, 0.0055323333..."
2,IterativeImputer_sklearn,k,-0.068082,0.112235,0.065940,1.658053e+06,51.964547,0.586598,334.8268,"[8.2e-08, 1.4e-08, 0.365, 0.297, 0.24924, 0.24...","[0.0006073222457892713, 0.3480874036765325, 0...."
3,KNN_3,k,0.013792,0.107847,0.075775,1.546109e+06,52.190638,0.674087,0.6088,"[8.2e-08, 1.4e-08, 0.365, 0.297, 0.24924, 0.24...","[0.19533333799999997, 0.2913333333333333, 0.28..."
4,KNN_3,lip,-0.470621,1.856507,1.217353,1.593333e+00,64.808363,0.787197,0.7479,"[2.1, 1.0, 1.47, 2.75, 2.6, 0.16, 0.1, 1.18, 1...","[2.8299999999999996, 0.7833333333333333, 0.933..."
5,KNN_4,lip,-0.315722,1.756015,1.215962,1.521342e+00,66.219926,0.786298,0.6468,"[2.1, 1.0, 1.47, 2.75, 2.6, 0.16, 0.1, 1.18, 1...","[2.3724999999999996, 1.2925, 0.825, 2.0925, 1...."
6,KNN_14,mg,-2.430788,0.022606,0.014036,3.246658e+00,65.466131,1.111056,0.3938,"[0.015988, 0.016113, 0.037, 0.022, 0.018805, 0...","[0.018586857142857145, 0.018777153846153846, 0..."
7,KNN_15,mg,-2.430788,0.022606,0.014036,3.246658e+00,65.466131,1.111056,0.4026,"[0.015988, 0.016113, 0.037, 0.022, 0.018805, 0...","[0.018586857142857145, 0.018777153846153846, 0..."
8,IterativeImputer_sklearn,mn,0.731459,0.000105,0.000062,4.036535e-01,45.629751,0.299106,263.1243,"[7.59e-05, 0.000396, 0.0002178, 0.000214, 0.00...","[2.639345219332818e-05, 0.00022612179142786953..."
9,KNN_3,mn,0.619093,0.000125,0.000084,4.527230e-01,50.965232,0.401985,1.0603,"[7.59e-05, 0.000396, 0.0002178, 0.000214, 0.00...","[0.00020926666666666667, 0.0002142666666666666..."


Display the top 3 imputers for each nutrient, sorted by column name and then SMAPE.

In [ ]:
df_results_onehot_cv.groupby("col_name").apply(lambda x: x.sort_values(["smape", "rmse"]).head(3)).reset_index(drop=True).sort_values(by=["col_name", "smape"], ascending=True)

,imputer_name,col_name,r2,rmse,mae,mape,smape,mae_std_ratio,mean_time,real_values,predicted_values
0,IterativeImputer_sklearn,fe,0.648056,0.003300,0.001860,1.133531e+00,61.964051,0.327668,232.9856,"[0.0019, 0.00321, 0.015131, 0.013225, 0.0038, ...","[0.003862832892899064, 0.0031234627212915825, ..."
1,KNN_3,fe,0.374481,0.004399,0.002799,1.574901e+00,65.486241,0.493079,0.6846,"[0.0019, 0.00321, 0.015131, 0.013225, 0.0038, ...","[0.0012613333333333333, 0.002369, 0.0055323333..."
2,KNN_4,fe,0.258884,0.004789,0.003078,1.567568e+00,68.165084,0.542077,0.6437,"[0.0019, 0.00321, 0.015131, 0.013225, 0.0038, ...","[0.0017885000000000002, 0.0025622500000000003,..."
3,IterativeImputer_sklearn,k,-0.068082,0.112235,0.065940,1.658053e+06,51.964547,0.586598,334.8268,"[8.2e-08, 1.4e-08, 0.365, 0.297, 0.24924, 0.24...","[0.0006073222457892713, 0.3480874036765325, 0...."
4,KNN_3,k,0.013792,0.107847,0.075775,1.546109e+06,52.190638,0.674087,0.6088,"[8.2e-08, 1.4e-08, 0.365, 0.297, 0.24924, 0.24...","[0.19533333799999997, 0.2913333333333333, 0.28..."
5,KNN_4,k,-0.169401,0.117438,0.081982,1.652642e+06,54.701298,0.729305,0.6793,"[8.2e-08, 1.4e-08, 0.365, 0.297, 0.24924, 0.24...","[0.21850000349999998, 0.30974999999999997, 0.2..."
6,KNN_3,lip,-0.470621,1.856507,1.217353,1.593333e+00,64.808363,0.787197,0.7479,"[2.1, 1.0, 1.47, 2.75, 2.6, 0.16, 0.1, 1.18, 1...","[2.8299999999999996, 0.7833333333333333, 0.933..."
7,KNN_4,lip,-0.315722,1.756015,1.215962,1.521342e+00,66.219926,0.786298,0.6468,"[2.1, 1.0, 1.47, 2.75, 2.6, 0.16, 0.1, 1.18, 1...","[2.3724999999999996, 1.2925, 0.825, 2.0925, 1...."
8,KNN_7,lip,-0.237950,1.703326,1.196677,1.580770e+00,66.399174,0.773827,0.8741,"[2.1, 1.0, 1.47, 2.75, 2.6, 0.16, 0.1, 1.18, 1...","[2.234285714285714, 1.1585714285714286, 0.8557..."
9,KNN_14,mg,-2.430788,0.022606,0.014036,3.246658e+00,65.466131,1.111056,0.3938,"[0.015988, 0.016113, 0.037, 0.022, 0.018805, 0...","[0.018586857142857145, 0.018777153846153846, 0..."


### 4.4. Select the Best Imputer for Each Nutrient
For each nutrient column, select the single best imputer based on the lowest SMAPE (primary metric) and RMSE (secondary metric if SMAPE is tied).

In [ ]:
# Select the best imputer for each column
best_imputers_per_col = df_results_onehot_cv.loc[df_results_onehot_cv.groupby("col_name")["smape"].idxmin()].reset_index(drop=True)
# Alternative: df_results_onehot_cv.groupby("col_name").apply(lambda x: x.sort_values(["smape", "rmse"]).head(1)).reset_index(drop=True)
best_imputers_per_col = best_imputers_per_col.sort_values("smape", ascending=True)
best_imputers_per_col

,imputer_name,col_name,r2,rmse,mae,mape,smape,mae_std_ratio,mean_time,real_values,predicted_values
6,KNN_15,ptn,-0.516015,3.738744,2.283795,1.025987e-01,11.063064,0.745117,0.4978,"[22.5, 23.5, 22.29, 24.0, 21.05, 21.54, 23.92,...","[22.179160000000003, 23.968493333333335, 23.61..."
4,IterativeImputer_sklearn,mn,0.731459,0.000105,0.000062,4.036535e-01,45.629751,0.299106,263.1243,"[7.59e-05, 0.000396, 0.0002178, 0.000214, 0.00...","[2.639345219332818e-05, 0.00022612179142786953..."
5,KNN_3,na,-10258.098236,4.074201,1.150600,1.059941e+02,49.024160,27.482453,0.5505,"[0.12, 0.125, 0.086821, 0.09539, 0.098926, 0.0...","[0.11166666666666665, 0.11166666666666665, 0.0..."
1,IterativeImputer_sklearn,k,-0.068082,0.112235,0.065940,1.658053e+06,51.964547,0.586598,334.8268,"[8.2e-08, 1.4e-08, 0.365, 0.297, 0.24924, 0.24...","[0.0006073222457892713, 0.3480874036765325, 0...."
0,IterativeImputer_sklearn,fe,0.648056,0.003300,0.001860,1.133531e+00,61.964051,0.327668,232.9856,"[0.0019, 0.00321, 0.015131, 0.013225, 0.0038, ...","[0.003862832892899064, 0.0031234627212915825, ..."
2,KNN_3,lip,-0.470621,1.856507,1.217353,1.593333e+00,64.808363,0.787197,0.7479,"[2.1, 1.0, 1.47, 2.75, 2.6, 0.16, 0.1, 1.18, 1...","[2.8299999999999996, 0.7833333333333333, 0.933..."
3,KNN_14,mg,-2.430788,0.022606,0.014036,3.246658e+00,65.466131,1.111056,0.3938,"[0.015988, 0.016113, 0.037, 0.022, 0.018805, 0...","[0.018586857142857145, 0.018777153846153846, 0..."
10,KNN_12,zn,-54.959702,0.011613,0.003038,1.196098e+01,69.509690,1.924304,0.6980,"[0.0024, 0.00235, 0.00495, 0.0033, 0.00429, 0....","[0.001939, 0.0024015, 0.0019362499999999998, 0..."
9,IterativeImputer_sklearn,w6,0.494885,0.292161,0.208613,3.087839e+03,82.386449,0.493938,261.2871,"[0.5712, 0.266, 5.39e-06, 1.57, 1.04, 0.000412...","[0.3915037116972342, 0.2759173079805953, 0.306..."
7,IterativeImputer_sklearn,se,-794.094740,0.001068,0.000328,7.716402e+00,83.628472,8.321785,334.8947,"[1.3e-05, 4e-06, 4.62e-06, 6.6e-06, 0.000132, ...","[1.2356582924873453e-05, 0.0, 8.29421316507393..."


## 5. Full Dataset Imputation

Using the best imputer identified for each nutrient column from the cross-validation step, impute the missing values in the entire dataset.

### 5.1. Define Function for Full Column Imputation

In [9]:
def impute_column_final(df_full_data, col_name_to_impute, imputer_model_name):
    """Imputes a single column in the DataFrame using the specified imputer.

    Args:
        df_full_data (pd.DataFrame): The one-hot encoded DataFrame with all original missing values.
        col_name_to_impute (str): The name of the column to impute.
        imputer_model_name (str): The name of the imputer to use.

    Returns:
        np.array: The imputed column (as a 1D NumPy array).
        float: Time taken for imputation.
    """
    df_copy = df_full_data.copy()  # Work on a copy
    col_idx = df_copy.columns.get_loc(col_name_to_impute)
    input_df_np = df_copy.values

    start_time = time.time()

    # Initialize the imputer (same logic as in impute_cv)
    if imputer_model_name.startswith("KNN"):
        k = int(imputer_model_name.split("_")[1])
        imputer = KNNImputer(n_neighbors=k)
    elif imputer_model_name == "MatrixFactorization":
        imputer = MatrixFactorization(verbose=False, min_value=0)
    elif imputer_model_name == "NuclearNormMinimization":
        imputer = NuclearNormMinimization(verbose=False, min_value=0)
    elif imputer_model_name == "SoftImpute":
        imputer = SoftImpute(verbose=False, min_value=0)
    elif imputer_model_name == "BiScaler+SoftImpute":
        imputer = SoftImpute(verbose=False, min_value=0)  # BiScaler applied separately
    elif imputer_model_name == "IterativeSVD":
        imputer = IterativeSVD(verbose=False, min_value=0)
    elif imputer_model_name == "IterativeImputer_sklearn":
        imputer = IterativeImputer_sklearn(verbose=0, min_value=0.0, random_state=314)
    elif imputer_model_name == "IterativeImputer_sklearn_LinearRegression":
        imputer = IterativeImputer_sklearn(verbose=0, estimator=LinearRegression(), min_value=0.0, random_state=314)
    # Add other imputers if they were top performers
    else:
        raise ValueError(f"Unknown imputer for final imputation: {imputer_model_name}")

    # Perform imputation
    if imputer_model_name == "BiScaler+SoftImpute":
        scaled_matrix = BiScaler(verbose=False).fit_transform(input_df_np)
        imputed_matrix = imputer.fit_transform(scaled_matrix)
    else:
        imputed_matrix = imputer.fit_transform(input_df_np)

    time_elapsed = time.time() - start_time
    print(f"Time elapsed for {col_name_to_impute} with {imputer_model_name}: {time_elapsed:.2f}s")
    return imputed_matrix[:, col_idx], time_elapsed

### 5.2. Apply Best Imputers to the Dataset

In [ ]:
data_onehot_imputed_final = data_onehot.copy()  # Start with the one-hot encoded data with original NaNs
best_imputer_pairs = best_imputers_per_col[["col_name", "imputer_name"]].values

for col_name, imputer_name_selected in best_imputer_pairs:
    print(f"Imputing column '{col_name}' with imputer '{imputer_name_selected}'")
    # Pass the full data_onehot which has NaNs to be imputed
    imputed_column_values, _ = impute_column_final(data_onehot.copy(), col_name, imputer_name_selected)
    data_onehot_imputed_final[col_name] = imputed_column_values

# Display the imputed DataFrame (optional)
data_onehot_imputed_final

Imputing ptn with KNN_15
Time elapsed: 0.007914304733276367
Imputing mn with IterativeImputer_sklearn
Time elapsed: 5.6133527755737305
Imputing na with KNN_3
Time elapsed: 0.012720108032226562
Imputing k with IterativeImputer_sklearn
Time elapsed: 6.187098026275635
Imputing fe with IterativeImputer_sklearn
Time elapsed: 5.6820666790008545
Imputing lip with KNN_3
Time elapsed: 0.009148120880126953
Imputing mg with KNN_14
Time elapsed: 0.011572837829589844
Imputing zn with KNN_12
Time elapsed: 0.0077588558197021484
Imputing w6 with IterativeImputer_sklearn
Time elapsed: 5.8762757778167725
Imputing se with IterativeImputer_sklearn
Time elapsed: 6.0794150829315186
Imputing w3 with IterativeSVD
Time elapsed: 0.35091161727905273


,fe,mn,se,zn,k,mg,na,ptn,lip,w3,w6,taxon_tfmed_AA,taxon_tfmed_AP1,taxon_tfmed_AP2,taxon_tfmed_BC,taxon_tfmed_CC1,taxon_tfmed_CC2,taxon_tfmed_CD,taxon_tfmed_CE,taxon_tfmed_CP1,taxon_tfmed_CP2,taxon_tfmed_CY,taxon_tfmed_DL,taxon_tfmed_EQ,taxon_tfmed_MA,taxon_tfmed_OH,taxon_tfmed_OV,taxon_tfmed_PA,taxon_tfmed_PD,taxon_tfmed_PE,taxon_tfmed_PQ,taxon_tfmed_PS,taxon_tfmed_PU,taxon_tfmed_SC,taxon_tfmed_SR,taxon_tfmed_SS,taxon_tfmed_ST,taxon_tfmed_TP1,taxon_tfmed_TP2,taxon_tfmed_TT,genus_agouti,genus_alces,genus_anas,genus_branta,genus_caiman,genus_capreolus,genus_cervus,genus_chelonoidis,genus_columba,genus_cuniculus,genus_dasyprocta,genus_equus,genus_manzama,genus_mazama,genus_odocoileus,genus_pecari,genus_peltocephalus,genus_phacochoerus,genus_podocnemis,genus_scolopax,genus_streptopelia,genus_sus,genus_syncerus,genus_tayassu,genus_turdus,classificacao_ave,classificacao_mamifero,classificacao_reptil,pais_coleta_Africa do Sul,pais_coleta_Alemanha,pais_coleta_Brasil,pais_coleta_Canadá,pais_coleta_EUA,pais_coleta_Espanha,pais_coleta_Estados Unidos da América,pais_coleta_Italia,pais_coleta_Itália,pais_coleta_Letónia,pais_coleta_Nova Zelândia,pais_coleta_Peru,pais_coleta_Polônia,pais_coleta_África do Sul,pais_coleta_África do sul,partes_tfmed_Musculo,partes_tfmed_Visceras
0,0.001900,0.000092,0.000013,0.002400,2.198771e-01,0.018157,0.111667,22.500000,2.100000,0.077700,0.571200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.003210,0.000059,0.000004,0.002350,2.395952e-01,0.019586,0.111667,23.500000,1.000000,0.110000,0.266000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.004041,0.000114,0.000010,0.001838,2.047861e-01,0.018157,0.111667,22.290000,1.470000,0.000032,0.000005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.003154,0.000076,0.000005,0.004950,2.611694e-01,0.018386,0.118000,23.380667,0.473333,0.108578,0.039212,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.013631,0.000396,0.000007,0.003300,1.054439e-01,0.019586,0.118000,23.521333,0.473333,0.195693,1.145260,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
5,0.013631,0.000218,0.000132,0.004290,1.054456e-01,0.019586,0.118000,23.698000,0.473333,0.195693,1.145258,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
6,0.015131,0.000214,0.000069,0.002040,8.200000e-08,0.015988,0.111667,23.698000,1.053333,0.195999,1.348317,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

### 5.3. Post-Imputation Adjustments and Saving
Nutrient values cannot be negative. Any imputed values less than 0 are set to 0. The fully imputed dataset and the imputation performance metrics are saved.

In [ ]:
# Ensure no nutrient value is negative after imputation
for col in cols_impute:  # Iterate only over nutrient columns
    if col in data_onehot_imputed_final.columns:
        data_onehot_imputed_final[col] = data_onehot_imputed_final[col].clip(lower=0)

# Save the imputed one-hot encoded data
data_onehot_imputed_final.to_csv("data/imputed_data.csv", index=False)
# Save the metrics of the best imputers
best_imputers_per_col.to_csv("data/imputation_metrics.csv", index=False)

In [12]:
# Load the saved imputed data for verification (optional)
data_onehot_imputed_final = pd.read_csv("data/imputed_data.csv")
data_onehot_imputed_final.head()

,fe,mn,se,zn,k,mg,na,ptn,lip,w3,w6,taxon_tfmed_AA,taxon_tfmed_AP1,taxon_tfmed_AP2,taxon_tfmed_BC,taxon_tfmed_CC1,taxon_tfmed_CC2,taxon_tfmed_CD,taxon_tfmed_CE,taxon_tfmed_CP1,taxon_tfmed_CP2,taxon_tfmed_CY,taxon_tfmed_DL,taxon_tfmed_EQ,taxon_tfmed_MA,taxon_tfmed_OH,taxon_tfmed_OV,taxon_tfmed_PA,taxon_tfmed_PD,taxon_tfmed_PE,taxon_tfmed_PQ,taxon_tfmed_PS,taxon_tfmed_PU,taxon_tfmed_SC,taxon_tfmed_SR,taxon_tfmed_SS,taxon_tfmed_ST,taxon_tfmed_TP1,taxon_tfmed_TP2,taxon_tfmed_TT,genus_agouti,genus_alces,genus_anas,genus_branta,genus_caiman,genus_capreolus,genus_cervus,genus_chelonoidis,genus_columba,genus_cuniculus,genus_dasyprocta,genus_equus,genus_manzama,genus_mazama,genus_odocoileus,genus_pecari,genus_peltocephalus,genus_phacochoerus,genus_podocnemis,genus_scolopax,genus_streptopelia,genus_sus,genus_syncerus,genus_tayassu,genus_turdus,classificacao_ave,classificacao_mamifero,classificacao_reptil,pais_coleta_Africa do Sul,pais_coleta_Alemanha,pais_coleta_Brasil,pais_coleta_Canadá,pais_coleta_EUA,pais_coleta_Espanha,pais_coleta_Estados Unidos da América,pais_coleta_Italia,pais_coleta_Itália,pais_coleta_Letónia,pais_coleta_Nova Zelândia,pais_coleta_Peru,pais_coleta_Polônia,pais_coleta_África do Sul,pais_coleta_África do sul,partes_tfmed_Musculo,partes_tfmed_Visceras
0,0.001900,0.000092,0.000013,0.002400,0.219877,0.018157,0.111667,22.500000,2.100000,0.077700,0.571200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.003210,0.000059,0.000004,0.002350,0.239595,0.019586,0.111667,23.500000,1.000000,0.110000,0.266000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.004041,0.000114,0.000010,0.001838,0.204786,0.018157,0.111667,22.290000,1.470000,0.000032,0.000005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.003154,0.000076,0.000005,0.004950,0.261169,0.018386,0.118000,23.380667,0.473333,0.108578,0.039212,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.013631,0.000396,0.000007,0.003300,0.105444,0.019586,0.118000,23.521333,0.473333,0.195693,1.145260,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


## 6. Post-Imputation Analysis and Export

This section focuses on preparing the imputed data for easier interpretation and for generating summary reports and visualizations. It involves:
1.  Reconstructing a 'user-friendly' version of the dataset by combining the imputed numerical nutrient data with the original (non-one-hot-encoded) categorical columns.
2.  Creating an Excel file where imputed values are highlighted for easy identification.
3.  Generating descriptive statistics for the imputed nutrients.
4.  Creating boxplots to visualize the distribution of each imputed nutrient.

### 6.1. Reconstruct User-Friendly Dataset and Highlight Imputed Values

In [13]:
# 'data' is the original processed data before one-hot encoding
data.head()

,taxon_tfmed,genus,classificacao,pais_coleta,fe,mn,se,zn,k,mg,na,ptn,lip,w3,w6,partes_tfmed,classificacao_cat,pais_coleta_cat,partes_tfmed_cat,taxon_tfmed_cat,genus_cat
0,SS,sus,mamifero,Alemanha,0.00190,NaN,0.000013,0.00240,NaN,NaN,NaN,22.50,2.10,0.077700,0.571200,Musculo,1,1,0,24,21
1,CC1,capreolus,mamifero,Alemanha,0.00321,NaN,0.000004,0.00235,NaN,NaN,NaN,23.50,1.00,0.110000,0.266000,Musculo,1,1,0,4,5
2,EQ,equus,mamifero,África do Sul,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.29,1.47,0.000032,0.000005,Musculo,1,14,0,12,11
3,CE,cervus,mamifero,Polônia,NaN,0.000076,0.000005,0.00495,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Musculo,1,13,0,7,6
4,CE,cervus,mamifero,Polônia,NaN,0.000396,0.000007,0.00330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Visceras,1,13,1,7,6


In [14]:
# Select original categorical columns (and any other non-imputed, non-onehot columns)
original_categorical_cols = [col for col in data.columns if col not in cols_impute and "_cat" not in col]
data_cats_original = data[original_categorical_cols].copy()

# Select only the imputed nutrient columns from the one-hot imputed data
data_imputed_nutrients = data_onehot_imputed_final[cols_impute].copy()

# Select original nutrient columns (with NaNs) for comparison
data_original_nutrients = data[cols_impute].copy()

data_imputed_nutrients.head()

,fe,mn,se,zn,k,mg,na,ptn,lip,w3,w6
0,0.001900,0.000092,0.000013,0.002400,0.219877,0.018157,0.111667,22.500000,2.100000,0.077700,0.571200
1,0.003210,0.000059,0.000004,0.002350,0.239595,0.019586,0.111667,23.500000,1.000000,0.110000,0.266000
2,0.004041,0.000114,0.000010,0.001838,0.204786,0.018157,0.111667,22.290000,1.470000,0.000032,0.000005
3,0.003154,0.000076,0.000005,0.004950,0.261169,0.018386,0.118000,23.380667,0.473333,0.108578,0.039212
4,0.013631,0.000396,0.000007,0.003300,0.105444,0.019586,0.118000,23.521333,0.473333,0.195693,1.145260


In [19]:
# Concatenate original categorical data with imputed nutrient data
data_imputed_user_friendly = pd.concat([data_cats_original, data_imputed_nutrients], axis=1)

# Concatenate original categorical data with original nutrient data (for mask creation)
data_original_user_friendly = pd.concat([data_cats_original, data_original_nutrients], axis=1)

# Create a mask to identify where original values were NaN (these are the imputed cells)
mask_imputed_cells = data_original_user_friendly[cols_impute].isnull()


# Function to apply conditional styling (highlight imputed cells)
def highlight_imputed(s, column_name, mask_df):
    is_imputed = mask_df[column_name]
    return ["background-color: red" if v else "" for v in is_imputed]


# Apply styling to the user-friendly imputed DataFrame
# We need to be careful with Styler apply, it works column-wise or row-wise or table-wise
styled_df = data_imputed_user_friendly.style
for col in cols_impute:
    styled_df = styled_df.apply(highlight_imputed, column_name=col, mask_df=mask_imputed_cells, subset=[col], axis=0)

# Export to Excel
styled_df.to_excel("data/imputed_data_highlighted.xlsx", engine="openpyxl", index=False)

In [20]:
# Display styled DataFrame (optional, can be large in notebook)
styled_df


,taxon_tfmed,genus,classificacao,pais_coleta,partes_tfmed,fe,mn,se,zn,k,mg,na,ptn,lip,w3,w6
0,SS,sus,mamifero,Alemanha,Musculo,0.001900,0.000092,0.000013,0.002400,0.219877,0.018157,0.111667,22.500000,2.100000,0.077700,0.571200
1,CC1,capreolus,mamifero,Alemanha,Musculo,0.003210,0.000059,0.000004,0.002350,0.239595,0.019586,0.111667,23.500000,1.000000,0.110000,0.266000
2,EQ,equus,mamifero,África do Sul,Musculo,0.004041,0.000114,0.000010,0.001838,0.204786,0.018157,0.111667,22.290000,1.470000,0.000032,0.000005
3,CE,cervus,mamifero,Polônia,Musculo,0.003154,0.000076,0.000005,0.004950,0.261169,0.018386,0.118000,23.380667,0.473333,0.108578,0.039212
4,CE,cervus,mamifero,Polônia,Visceras,0.013631,0.000396,0.000007,0.003300,0.105444,0.019586,0.118000,23.521333,0.473333,0.195693,1.145260
5,CE,cervus,mamifero,Polônia,Visceras,0.013631,0.000218,0.000132,0.004290,0.105446,0.019586,0.118000,23.698000,0.473333,0.195693,1.145258
6,CE,cervus,mamifero,EUA,Visceras,0.015131,0.000214,0.000069,0.002040,0.000000,0.015988,0.111667,23.698000,1.053333,0.195999,1.348317
7,CE,cervus,mamifero,EUA,Visceras,0.013225,0.000211,0.000069,0.001924,0.000000,0.016113,0.111667,23.614000,1.053333,0.195999,1.348220
8,SR,scolopax,ave,Italia,Musculo,0.004821,0.000143,0.000002,0.001811,0.264851,0.017086,0.076921,24.000000,2.750000,0.000197,1.570000
9,SR,scolopax,ave,Italia,Musculo,0.006687,0.000178,0.000026,0.001704,0.210983,0.018157,0.111667,21.050000,2.600000,0.000175,1.040000


Show the mask indicating imputed cells.

In [21]:
mask_imputed_cells.head()

,fe,mn,se,zn,k,mg,na,ptn,lip,w3,w6
0,False,True,False,False,True,True,True,False,False,False,False
1,False,True,False,False,True,True,True,False,False,False,False
2,True,True,True,True,True,True,True,False,False,False,False
3,True,False,False,False,True,True,True,True,True,True,True
4,True,False,False,False,True,True,True,True,True,True,True


Show the original data for comparison.

In [22]:
data_original_user_friendly.head()

,taxon_tfmed,genus,classificacao,pais_coleta,partes_tfmed,fe,mn,se,zn,k,mg,na,ptn,lip,w3,w6
0,SS,sus,mamifero,Alemanha,Musculo,0.00190,NaN,0.000013,0.00240,NaN,NaN,NaN,22.50,2.10,0.077700,0.571200
1,CC1,capreolus,mamifero,Alemanha,Musculo,0.00321,NaN,0.000004,0.00235,NaN,NaN,NaN,23.50,1.00,0.110000,0.266000
2,EQ,equus,mamifero,África do Sul,Musculo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.29,1.47,0.000032,0.000005
3,CE,cervus,mamifero,Polônia,Musculo,NaN,0.000076,0.000005,0.00495,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CE,cervus,mamifero,Polônia,Visceras,NaN,0.000396,0.000007,0.00330,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 6.2. Export Imputation Metrics
Clean up and save the imputation metrics for the best performing models.

In [23]:
best_imputers_metrics_final = pd.read_csv("data/imputation_metrics.csv")
best_imputers_metrics_final

,imputer_name,col_name,r2,rmse,mae,mape,smape,mae_std_ratio,mean_time,real_values,predicted_values
0,KNN_15,ptn,-0.516015,3.738744,2.283795,1.025987e-01,11.063064,0.745117,0.4978,[22.5 23.5 22.29 24. 21.05 21.54...,[22.17916 23.96849333 23.61049333 21.982333...
1,IterativeImputer_sklearn,mn,0.731459,0.000105,0.000062,4.036535e-01,45.629751,0.299106,263.1243,[7.590e-05 3.960e-04 2.178e-04 2.140e-04 2.110...,[2.63934522e-05 2.26121791e-04 2.53408961e-04 ...
2,KNN_3,na,-10258.098236,4.074201,1.150600,1.059941e+02,49.024160,27.482453,0.5505,[0.12 0.125 0.086821 0.09539 0.098...,[1.11666667e-01 1.11666667e-01 9.58863333e-02 ...
3,IterativeImputer_sklearn,k,-0.068082,0.112235,0.065940,1.658053e+06,51.964547,0.586598,334.8268,[8.200000e-08 1.400000e-08 3.650000e-01 2.9700...,[0.00060732 0.3480874 0.25317084 0.28968947 0...
4,IterativeImputer_sklearn,fe,0.648056,0.003300,0.001860,1.133531e+00,61.964051,0.327668,232.9856,[0.0019 0.00321 0.015131 0.013225 0.0038 ...,[3.86283289e-03 3.12346272e-03 1.31813626e-02 ...
5,KNN_3,lip,-0.470621,1.856507,1.217353,1.593333e+00,64.808363,0.787197,0.7479,[2.1 1. 1.47 2.75 2.6 0.16 0.1 1.18 1.12 ...,[2.83 0.78333333 0.93333333 2.75666667 2...
6,KNN_14,mg,-2.430788,0.022606,0.014036,3.246658e+00,65.466131,1.111056,0.3938,[0.015988 0.016113 0.037 0.022 0.018805 ...,[0.01858686 0.01877715 0.01725858 0.01682755 0...
7,KNN_12,zn,-54.959702,0.011613,0.003038,1.196098e+01,69.509690,1.924304,0.6980,[0.0024 0.00235 0.00495 0.0033 0...,[0.001939 0.0024015 0.00193625 0.00263083 0...
8,IterativeImputer_sklearn,w6,0.494885,0.292161,0.208613,3.087839e+03,82.386449,0.493938,261.2871,[5.71200e-01 2.66000e-01 5.39000e-06 1.57000e+...,[0.39150371 0.27591731 0.30666756 1.06367137 0...
9,IterativeImputer_sklearn,se,-794.094740,0.001068,0.000328,7.716402e+00,83.628472,8.321785,334.8947,[1.30e-05 4.00e-06 4.62e-06 6.60e-06 1.32e-04 ...,[1.23565829e-05 0.00000000e+00 8.29421317e-05 ...


In [24]:
# Drop columns not needed for the final metrics report in the paper (e.g., raw prediction arrays)
cols_to_drop_from_metrics = ["r2", "real_values", "predicted_values", "mape", "mean_time"]
best_imputers_metrics_cleaned = best_imputers_metrics_final.drop(columns=cols_to_drop_from_metrics, errors="ignore")

best_imputers_metrics_cleaned.to_excel("data/final_imputation_metrics_summary.xlsx", engine="openpyxl", index=False)
best_imputers_metrics_cleaned

,imputer_name,col_name,rmse,mae,smape,mae_std_ratio
0,KNN_15,ptn,3.738744,2.283795,11.063064,0.745117
1,IterativeImputer_sklearn,mn,0.000105,0.000062,45.629751,0.299106
2,KNN_3,na,4.074201,1.150600,49.024160,27.482453
3,IterativeImputer_sklearn,k,0.112235,0.065940,51.964547,0.586598
4,IterativeImputer_sklearn,fe,0.003300,0.001860,61.964051,0.327668
5,KNN_3,lip,1.856507,1.217353,64.808363,0.787197
6,KNN_14,mg,0.022606,0.014036,65.466131,1.111056
7,KNN_12,zn,0.011613,0.003038,69.509690,1.924304
8,IterativeImputer_sklearn,w6,0.292161,0.208613,82.386449,0.493938
9,IterativeImputer_sklearn,se,0.001068,0.000328,83.628472,8.321785


### 6.3. Generate Summary Statistics

In [25]:
# Overall summary statistics for imputed nutrient columns
summary_stats_overall = data_imputed_user_friendly[cols_impute].describe()
# summary_stats_overall
summary_stats_overall.to_excel("data/summary_statistics_overall_imputed.xlsx")

In [26]:
# Summary statistics per anatomical part ('partes_tfmed')
summary_stats_per_part = data_imputed_user_friendly.groupby("partes_tfmed")[cols_impute].describe()
# summary_stats_per_part
summary_stats_per_part.to_excel("data/summary_statistics_per_part_imputed.xlsx")

In [27]:
# Summary statistics per taxonomic class ('classificacao')
summary_stats_per_class = data_imputed_user_friendly.groupby("classificacao")[cols_impute].describe()
# summary_stats_per_class
summary_stats_per_class.to_excel("data/summary_statistics_per_class_imputed.xlsx")

In [28]:
# Summary statistics per taxonomic class and anatomical part
summary_stats_class_part = data_imputed_user_friendly.groupby(by=["classificacao", "partes_tfmed"])[cols_impute].describe()
# summary_stats_class_part
summary_stats_class_part.to_excel("data/summary_statistics_per_class_and_part_imputed.xlsx")

### 6.4. Generate Boxplots for Imputed Nutrients

In [29]:
import plotly.express as px
import os

# Ensure the directory for boxplots exists
output_boxplot_dir = "data/boxplots_imputed"
os.makedirs(output_boxplot_dir, exist_ok=True)

data_for_plots = data_imputed_user_friendly[cols_impute]

# Create a boxplot for each imputed nutrient feature
for feature in data_for_plots.columns:
    fig = px.box(data_for_plots, y=feature, title=f"Distribution of Imputed {feature.upper()}")
    fig.write_html(os.path.join(output_boxplot_dir, f"boxplot_{feature}.html"))

print(f"Boxplots saved to {output_boxplot_dir}")

Boxplots saved to data/boxplots_imputed


The data imputation process is now complete. The `imputed_data.csv` contains the fully imputed dataset in one-hot encoded form, while `imputed_data_highlighted.xlsx` provides a user-friendly view with imputed cells marked. Summary statistics and performance metrics are also saved for reporting.